# AI Recruiter Assistant

A conversational chatbot to pre-screen job offers from recruiters.

## Project Overview
- **Goal**: Automate initial screening of job offers
- **Technology**: Fine-tuned open-source LLM with RAG pipeline
- **Interface**: Gradio web application
- **Timeline**: 1 week development

## Architecture
1. Intent Detection → 2. RAG Analysis → 3. Match Scoring → 4. State Management → 5. Response Generation

## 1. Environment Setup & Dependencies

In [1]:
# Install required packages
!pip install transformers>=4.36.0 accelerate>=0.24.0 bitsandbytes>=0.41.0 torch>=2.0.0 peft>=0.7.0
!pip install langchain>=0.1.0 langchain-community>=0.0.10 faiss-cpu>=1.7.4 sentence-transformers>=2.2.0
!pip install gradio>=4.0.0 pandas>=2.0.0 numpy>=1.24.0 tqdm>=4.65.0 datasets>=2.14.0
!pip install scikit-learn>=1.3.0

In [2]:
# Import libraries
import os
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from enum import Enum

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

# Transformers imports
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType

# Gradio for web interface
import gradio as gr

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## 2. Data Loading & Processing

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
absolute_path = "/content/drive/MyDrive/Colab Notebooks/KEEPCODING/PROJECT/AI_Recruiter_Assistant"

# Load CV and job expectations
def load_documents():
    """Load CV and job expectations documents"""
    rag_path = os.path.join(absolute_path, "RAG")
    try:
        with open(f'{rag_path}/cv.md', 'r', encoding='utf-8') as f:
           cv_content = f.read()

        with open(f'{rag_path}/job_expectations.md', 'r', encoding='utf-8') as f:
            expectations_content = f.read()

        print(f"✅ CV loaded: {len(cv_content)} characters")
        print(f"✅ Job expectations loaded: {len(expectations_content)} characters")

        return cv_content, expectations_content
    except FileNotFoundError as e:
        print(f"❌ Error loading documents: {e}")
        return None, None

# Load LinkedIn messages for fine-tuning
def load_linkedin_messages():
    """Load LinkedIn messages dataset for fine-tuning"""
    data_path = os.path.join(absolute_path, "data")
    try:
        df = pd.read_csv(f'{data_path}/linkedin_messages.csv')
        print(f"✅ LinkedIn messages loaded: {len(df)} messages")
        print(f"Columns: {list(df.columns)}")
        return df
    except Exception as e:
        print(f"❌ Error loading LinkedIn messages: {e}")
        return None

# Load all data
cv_content, expectations_content = load_documents()
linkedin_df = load_linkedin_messages()

✅ CV loaded: 7032 characters
✅ Job expectations loaded: 72 characters
✅ LinkedIn messages loaded: 4739 messages
Columns: ['CONVERSATION ID', 'CONVERSATION TITLE', 'FROM', 'SENDER PROFILE URL', 'TO', 'RECIPIENT PROFILE URLS', 'DATE', 'SUBJECT', 'CONTENT', 'FOLDER']


## 3. RAG Pipeline Setup

In [10]:
# Create documents for RAG
def create_documents(cv_content: str, expectations_content: str) -> List[Document]:
    """Create LangChain documents from CV and expectations"""
    documents = []

    # Add CV document
    documents.append(Document(
        page_content=cv_content,
        metadata={"source": "cv", "type": "profile"}
    ))

    # Add job expectations document
    documents.append(Document(
        page_content=expectations_content,
        metadata={"source": "expectations", "type": "preferences"}
    ))

    return documents

# Split documents into chunks
def split_documents(documents: List[Document]) -> List[Document]:
    """Split documents into smaller chunks for better retrieval"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"✅ Documents split into {len(split_docs)} chunks")
    return split_docs

# Create vector database
def create_vector_db(documents: List[Document]):
    """Create FAISS vector database from documents"""
    # Use a lightweight embedding model
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )

    # Create vector store
    vector_db = FAISS.from_documents(documents, embeddings)
    print("✅ Vector database created successfully!")

    return vector_db, embeddings

# Initialize RAG pipeline
if cv_content and expectations_content:
    documents = create_documents(cv_content, expectations_content)
    split_docs = split_documents(documents)
    vector_db, embeddings = create_vector_db(split_docs)
else:
    print("❌ Cannot create RAG pipeline without documents")
    vector_db, embeddings = None, None

✅ Documents split into 10 chunks


/tmp/ipython-input-10-719529524.py:37: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Vector database created successfully!


## 4. State Management System

In [11]:
# Define conversation states
class ConversationState(Enum):
    PENDING_DETAILS = "pending_details"
    PASSED = "passed"
    STAND_BY = "stand_by"
    FINISHED = "finished"

@dataclass
class ConversationContext:
    state: ConversationState
    match_score: Optional[float] = None
    job_details: Optional[str] = None
    conversation_history: List[Dict] = None

    def __post_init__(self):
        if self.conversation_history is None:
            self.conversation_history = []

class ConversationManager:
    def __init__(self):
        self.context = ConversationContext(state=ConversationState.PENDING_DETAILS)

    def update_state(self, new_state: ConversationState, **kwargs):
        """Update conversation state and context"""
        self.context.state = new_state
        for key, value in kwargs.items():
            setattr(self.context, key, value)

    def add_message(self, role: str, content: str):
        """Add message to conversation history"""
        self.context.conversation_history.append({
            "role": role,
            "content": content,
            "timestamp": pd.Timestamp.now().isoformat()
        })

    def get_context_summary(self) -> str:
        """Get summary of current conversation context"""
        return f"State: {self.context.state.value}, Match Score: {self.context.match_score}"

# Initialize conversation manager
conversation_manager = ConversationManager()
print("✅ Conversation manager initialized!")

✅ Conversation manager initialized!


## 5. Intent Detection & Match Scoring

In [12]:
# Intent detection function
def detect_intent(message: str) -> str:
    """Detect if message is a job offer or generic message"""
    # Simple keyword-based detection (will be enhanced with LLM)
    job_keywords = [
        'position', 'role', 'job', 'opportunity', 'vacancy', 'opening',
        'salary', 'remote', 'hybrid', 'onsite', 'tech stack', 'requirements',
        'experience', 'years', 'skills', 'technologies', 'framework'
    ]

    generic_keywords = [
        'hi', 'hello', 'interested', 'opportunities', 'open', 'available',
        'connect', 'network', 'profile', 'background'
    ]

    message_lower = message.lower()

    job_score = sum(1 for keyword in job_keywords if keyword in message_lower)
    generic_score = sum(1 for keyword in generic_keywords if keyword in message_lower)

    if job_score > generic_score and job_score >= 2:
        return "job_offer"
    else:
        return "generic_message"

# Match scoring function
def calculate_match_score(job_description: str, vector_db, embeddings) -> float:
    """Calculate match score between job offer and profile"""
    if vector_db is None:
        return 0.0

    # Search for relevant documents
    relevant_docs = vector_db.similarity_search(job_description, k=5)

    # Calculate similarity scores
    job_embedding = embeddings.embed_query(job_description)

    scores = []
    for doc in relevant_docs:
        doc_embedding = embeddings.embed_query(doc.page_content)
        # Calculate cosine similarity
        similarity = np.dot(job_embedding, doc_embedding) / (
            np.linalg.norm(job_embedding) * np.linalg.norm(doc_embedding)
        )
        scores.append(similarity)

    # Return average score as percentage
    avg_score = np.mean(scores) if scores else 0.0
    return min(avg_score * 100, 100.0)  # Convert to percentage

print("✅ Intent detection and match scoring functions ready!")

✅ Intent detection and match scoring functions ready!


## 6. LLM Setup (Next Steps)

In the next cells, we'll:
1. Load the open-source LLM
2. Prepare LinkedIn messages for fine-tuning
3. Implement LoRA fine-tuning
4. Create response generation functions
5. Build the Gradio interface

Let's continue with the LLM setup...

In [ ]:
# Load base LLM (we'll use Mistral-7B-Instruct for good performance/size balance)
def load_base_model():
    """Load the base LLM model"""
    model_name = "mistralai/Mistral-7B-Instruct-v0.2"

    # Quantization config for memory efficiency
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="float16",
        bnb_4bit_use_double_quant=False,
    )

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    print(f"✅ Base model {model_name} loaded successfully!")
    return model, tokenizer

# Note: This will take some time to download the model
# Uncomment when ready to load the model
# base_model, base_tokenizer = load_base_model()

## 7. LinkedIn Messages Processing for Fine-tuning

In [ ]:
# Process LinkedIn messages for fine-tuning
def prepare_fine_tuning_data(linkedin_df: pd.DataFrame) -> List[Dict]:
    """Prepare LinkedIn messages for LoRA fine-tuning"""
    if linkedin_df is None:
        print("❌ No LinkedIn messages available for fine-tuning")
        return []

    # Display sample of the data
    print("Sample LinkedIn messages:")
    print(linkedin_df.head())

    # This function will be customized based on the actual structure of your CSV
    # For now, we'll create a template structure

    training_data = []

    # Example processing (adjust based on your actual CSV structure)
    for idx, row in linkedin_df.iterrows():
        # Assuming columns like 'message', 'response', 'context'
        # Adjust these column names based on your actual data

        if 'message' in row and 'response' in row:
            training_example = {
                "instruction": f"You are an AI recruiter assistant. Respond to this message: {row['message']}",
                "input": "",
                "output": row['response']
            }
            training_data.append(training_example)

    print(f"✅ Prepared {len(training_data)} training examples")
    return training_data

# Prepare training data
training_data = prepare_fine_tuning_data(linkedin_df)

## 8. Response Generation Functions

In [ ]:
# Response generation based on state and match score
def generate_response(
    message: str,
    state: ConversationState,
    match_score: Optional[float] = None
) -> str:
    """Generate appropriate response based on conversation state"""

    if state == ConversationState.PENDING_DETAILS:
        return """Thank you for reaching out! I'm interested in hearing more about this opportunity.
Could you please provide more details about:
• The specific role and responsibilities
• Required skills and experience level
• Tech stack and technologies used
• Salary range and benefits
• Work arrangement (remote/hybrid/onsite)
• Company culture and team size

This will help me better understand if this aligns with my career goals."""

    elif state == ConversationState.PASSED:
        return f"""Excellent! This opportunity looks like a great fit with a {match_score:.1f}% match to my profile.

I'm very interested in moving forward. Could you please let me know:
• Your availability for a call (preferred times)
• Preferred call duration (15 or 30 minutes)
• The objective of the call (get-to-know, technical interview, etc.)

I'm flexible and can accommodate your schedule."""

    elif state == ConversationState.STAND_BY:
        return f"""Thank you for sharing this opportunity! It seems interesting with a {match_score:.1f}% match to my profile.

I'd like to review the details more carefully before proceeding. I'll get back to you within 24-48 hours with my decision.

In the meantime, feel free to share any additional information that might be relevant."""

    elif state == ConversationState.FINISHED:
        return f"""Thank you for considering me for this opportunity! After reviewing the details,
I don't think this is the right fit for me at the moment (match score: {match_score:.1f}%).

I'm currently looking for:
• Data Science and AI/ML roles with Python and cloud technologies
• Product-focused companies with innovative projects
• Remote or hybrid work arrangements
• Competitive salary packages

I'd be happy to stay connected for future opportunities that might be a better match.
Thank you again for reaching out!"""

    else:
        return "I'm processing your message. Please wait..."

print("✅ Response generation functions ready!")

## 9. Main Processing Function

In [ ]:
# Main processing function
def process_message(message: str) -> Tuple[str, ConversationState, float]:
    """Main function to process incoming messages"""

    # Add message to conversation history
    conversation_manager.add_message("recruiter", message)

    # Step 1: Intent Detection
    intent = detect_intent(message)

    if intent == "generic_message" and conversation_manager.context.state == ConversationState.PENDING_DETAILS:
        # Stay in pending_details state
        response = generate_response(message, ConversationState.PENDING_DETAILS)
        conversation_manager.add_message("assistant", response)
        return response, ConversationState.PENDING_DETAILS, 0.0

    elif intent == "job_offer" or conversation_manager.context.state != ConversationState.PENDING_DETAILS:
        # Step 2: Calculate match score
        match_score = calculate_match_score(message, vector_db, embeddings)

        # Step 3: Decision logic
        if match_score > 80:
            new_state = ConversationState.PASSED
        elif match_score >= 60:
            new_state = ConversationState.STAND_BY
        else:
            new_state = ConversationState.FINISHED

        # Update conversation state
        conversation_manager.update_state(new_state, match_score=match_score, job_details=message)

        # Generate response
        response = generate_response(message, new_state, match_score)
        conversation_manager.add_message("assistant", response)

        return response, new_state, match_score

    else:
        # Fallback response
        response = "I'm here to help! Please share the job details so I can assist you better."
        conversation_manager.add_message("assistant", response)
        return response, ConversationState.PENDING_DETAILS, 0.0

print("✅ Main processing function ready!")

## 10. Gradio Interface (Coming Soon)

In the next section, we'll create the Gradio web interface for the chatbot.

## Next Steps:

1. **Load the base LLM** (uncomment the model loading cell)
2. **Process LinkedIn messages** for fine-tuning
3. **Implement LoRA fine-tuning**
4. **Create Gradio interface**
5. **Test and deploy**

Would you like to proceed with loading the LLM and implementing the fine-tuning?

In [7]:
import os

# Get the current working directory
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# List files in the RAG directory
rag_dir = 'RAG'
if os.path.exists(rag_dir):
    print(f"\nFiles in '{rag_dir}':")
    for file in os.listdir(rag_dir):
        print(file)
else:
    print(f"\nDirectory '{rag_dir}' not found.")

# List files in the data directory
data_dir = 'data'
if os.path.exists(data_dir):
    print(f"\nFiles in '{data_dir}':")
    for file in os.listdir(data_dir):
        print(file)
else:
    print(f"\nDirectory '{data_dir}' not found.")

Current working directory: /content

Directory 'RAG' not found.

Directory 'data' not found.
